# Interactive Molecular Dynamics with OpenMM and NanoVer 

In this notebook, we run a pre-prepared OpenMM simulation of a methane molecule and a carbon nanotube, serving it for interactive molecular dynamics using NanoVer. 

We'll then connect a client running NGLView to it and apply some interactive forces directly from the notebook

In [ ]:
from nanover.omni import OmniRunner
from nanover.omni.openmm import OpenMMSimulation

## Run the NanoVer server

First, we need to create an OpenMMSimulation for our methane and nanotube system. 

Let's do so by loading in a pre-prepared NanoVer OpenMM XML file, which contains:
- the PDB file
- the OpenMM XML file, containing:
  - the System
  - the Integrator

This format ensures that the whole simulation can be specified in a single file. 

The `OpenMMSimulation` class takes the pre-prepared input file and creates a simulation object that can then be served for interactive molecular dynamics using OpenMM. For an example of a more advanced setup, see [our neuraminidase notebook](./openmm_neuraminidase.ipynb).

In [2]:
input_xml = "openmm_files/nanotube.xml"
nanotube_simulation = OpenMMSimulation.from_xml_path(input_xml)

Now we have our simulation ready, we can set up our NanoVer server to run the simulation. We run interactive OpenMM simulations in NanoVer using the `OmniRunner` class, which is in charge of serving and running the simulation.

In [13]:
# This will just automatically close previous instances of the runner if you re-run this cell multiple times
try:
    imd_runner.close()
except NameError:
    pass
imd_runner = OmniRunner.with_basic_server(nanotube_simulation, name="nanotube-omm-server")

In [14]:
print(f'{imd_runner.app_server.name}: serving at {imd_runner.app_server.address}:{imd_runner.app_server.port}')

nanotube-omm-simulation: serving at [::]:38801


All set up! All that remains is to run the simulation using the `.next()` command, which tells the runner to run the next simulation. In our case we only have one simulation loaded, but in general the `OmniRunner` class is capable of handling multiple simulations and/or recordings of simulations.

In [25]:
imd_runner.next()

The server is running, so you can connect to it in VR and you'll see something like this:

![nanotube](./images/nanover_nanotube.png)



## Visualizing the simulation with NGLView 

We have a little [script](nglclient.py) (which will eventually become part of NanoVer properly) which can show a NanoVer trajectory with [NGLView](https://github.com/arose/nglview)

To install NGLView: 

```
conda install nglview -c conda-forge
# might need: jupyter-nbextension enable nglview --py --sys-prefix

# if you already installed nglview, you can `upgrade`
conda upgrade nglview --force
# might need: jupyter-nbextension enable nglview --py --sys-prefix
```

Your browser needs to support WebGL (try Firefox or Edge)

In [16]:
from nglclient import NGLClient

In [17]:
client = NGLClient.connect_to_single_server(port=imd_runner.app_server.port)

/opt/homebrew/Caskroom/miniforge/base/envs/nanover-dev/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/opt/homebrew/Caskroom/miniforge/base/envs/nanover-dev/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/opt/homebrew/Caskroom/miniforge/base/envs/nanover-dev/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'elements' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/opt/homebrew/Caskroom/miniforge/base/envs/nanover-dev/lib/python3.12/site-packages/MDAnalysis/coordinates/PDB.py:1153: UserWarning: Found no information for attr: 'record_types' Using default value of 'ATOM'
  warnings.warn("Found no in

In [18]:
client.view

NGLWidget()

## Apply an Interactive Force 

Generally, you probably want to apply forces from the VR app, NanoVer iMD - but you can do it from python! 

First, let's apply a couple of forces to hold to nanotube in place (there are better ways to do this) 

In [19]:
from nanover.imd.particle_interaction import ParticleInteraction
import numpy as np

First, we ask the server for an ID for our interactions

In [20]:
anchor1_id = client.start_interaction()

In [21]:
anchor2_id = client.start_interaction()

Now, we can create two interactions, one at each end of the nanotube, and assign a position. 
You can repeatedly run the cell below, changing the positions or strength of the interactions. See if you can catch the methane! 

**Disclaimer**: This is *much* easier in VR.

In [22]:
first_position = np.array((10,1,0)) # nanometers!
second_position = np.array((10,1,0))
anchor1 = ParticleInteraction(position=first_position, scale=10, particles=(0,), interaction_type='spring')
anchor2 = ParticleInteraction(position=second_position, scale=10, particles=(59,), interaction_type='spring')
client.update_interaction(anchor1_id, anchor1)
client.update_interaction(anchor2_id, anchor2)

When you want to stop interacting with the simulation, simply terminate your interactions by running the two cells below.

In [23]:
client.stop_interaction(anchor1_id)

True

In [24]:
client.stop_interaction(anchor2_id)

True

As always, we should clean up after ourselves:

## Close the Server

In [27]:
imd_runner.close()

## Next Steps

* Set up an OpenMM simulation with an AMBER file for a [protein-ligand system](openmm_neuraminidase.ipynb) and simulate it in NanoVer.